In [96]:
import os
from collections import Counter

import nltk

file_dir = '../data/processed'
file_name = 'oshhamaho.txt'
# file_name = 'apkbr_ru.txt'
# file_name = 'elgkbr_ru.txt'
file_path = os.path.join(file_dir, file_name)

BUF_SIZE = 100000
cnt = Counter()
with open(file_path) as f:
    tmp_raw = f.read(BUF_SIZE)
    while tmp_raw:
        tokens = nltk.word_tokenize(tmp_raw)
        text = nltk.Text(tokens)
        clc = text.collocation_list(5000)
        cnt.update(clc)
        tmp_raw = f.read(BUF_SIZE)

In [97]:
import pandas as pd

data = [
    {
        'w_bigram': ' '.join(bigram),
        'freq': freq
    }
    for bigram, freq in cnt.items()
    if freq > 2
]

df = pd.DataFrame(data)
df = df.sort_values('freq', ascending=False)

In [98]:
df

,w_bigram,freq
0,нэхърэ нэхъ,216
67,сыт хуэдэ,213
42,сыт щыгъуи,209
93,куэд щIауэ,149
116,фIы дыдэу,131
...,...,...
1790,сабийм хуэдэу,3
1789,лъэпкъ цIыкIум,3
1786,гъащIэ гъуэгуанэр,3
1784,япэ секретару,3


In [99]:
# для дальнейшей работы сохраним полученные данные в csv файл
df.to_csv(
    os.path.join('../data/processed/word_bigrams', file_name.replace('.txt', '.csv')),
    index=False,
)

In [100]:
from nltk.probability import ConditionalFreqDist, ConditionalProbDist, MLEProbDist

cfdist = ConditionalFreqDist()

for bigram, freq in cnt.items():
    word1, word2 = bigram
    cfdist[word1][word2] = freq

cpdist = ConditionalProbDist(cfdist, MLEProbDist)


In [115]:
def get_prob(word1):
    prob_dict = {}
    for word_2 in cpdist[word1].samples():
        prob_dict[word_2] = cpdist[word1].prob(word_2) * 100

    return prob_dict


In [123]:
sorted(get_prob('псалъэ').items(), key=lambda x: x[1], reverse=True)

[('гуапэ', 14.814814814814813),
 ('дахэ', 9.25925925925926),
 ('быдэ', 6.481481481481481),
 ('шэрыуэхэмрэ', 5.555555555555555),
 ('куэд', 4.62962962962963),
 ('къызэринэкIащ', 2.7777777777777777),
 ('шэрыуэ', 1.8518518518518516),
 ('закъуэкIэ', 1.8518518518518516),
 ('шэрыуэхэр', 1.8518518518518516),
 ('пэжхэр', 1.8518518518518516),
 ('закъуэм', 1.8518518518518516),
 ('лей', 1.8518518518518516),
 ('мыжьэ-мывэм', 1.8518518518518516),
 ('дыдж', 1.8518518518518516),
 ('закъуэ', 1.8518518518518516),
 ('дыджхэр', 1.8518518518518516),
 ('гъэщIэрэщIэн', 1.8518518518518516),
 ('жимыIэу', 1.8518518518518516),
 ('гуемыIухэр', 0.9259259259259258),
 ('зэхэлъыр', 0.9259259259259258),
 ('жаIакъым', 0.9259259259259258),
 ('къыфIигъэкIакъым', 0.9259259259259258),
 ('халъхьащ', 0.9259259259259258),
 ('зэкIэлъыкIуэкIэм', 0.9259259259259258),
 ('зэпхахэм', 0.9259259259259258),
 ('зэкIэлъыкIуэкIэр', 0.9259259259259258),
 ('лъэпкъыгъуэм', 0.9259259259259258),
 ('гуэрхэр', 0.9259259259259258),
 ('кIапэр', 0

```python
[('гуапэ', 14.814814814814813),
 ('дахэ', 9.25925925925926),
 ('быдэ', 6.481481481481481),
 ('шэрыуэхэмрэ', 5.555555555555555),
 ('куэд', 4.62962962962963),
 ('къызэринэкIащ', 2.7777777777777777),
 ...
]
```
Мы здесь видим, что самые вероятные слова после `псалъэ` это `гуапэ`, `дахэ`, `быдэ` и т.д.

как мы можем использовать полученные данные?
- для определения того, какие слова чаще всего встречаются вместе
- для предсказания следующего слова с учетом предыдущего
чем это может быть полезно?
- автодополнение при вводе текста
- для автоматического исправления опечаток


Подобное можно еще сделать по токенам, а не по словам.
Тогда мы можем получить вариации слов с учетом падежей, числа и т.д.
То есть зная начало слова, мы можем предсказать наиболее вероятные варианты слова.
Что позволит более точно делать автодополнение и исправление опечаток.